In [1]:
import re
from datetime import datetime
import pandas as pd
import numpy as np
from tkinter import filedialog
from tkinter import *
import os, sys
import glob
from matplotlib.backends.backend_pdf import PdfPages
import time

In [2]:
# Tworzymy listę i dwa puste DF
listOfFiles = []
DFL = pd.DataFrame()
DFM = pd.DataFrame()

In [3]:
# Funkcja pobiera listę ścieżek
def get_filenames():
    Tk().withdraw()
    print("Initializing Dialogue... \nPlease select a file.")
    tk_filenames = filedialog.askdirectory()
    tempDir = tk_filenames
    return tempDir


choosen_dir = get_filenames()
# glob zbiera listę plików 
os.chdir(choosen_dir)
for file in glob.glob("*.csv"):
    listOfFiles.append(file)

# Do wcześniej stworzonych DF dodajemy kolejno pliki w zależności od tego na jaką literę się plik zaczyna.
for file in listOfFiles:
    if file.startswith('L'):
        Table = pd.read_csv(file, sep=',')
        DF1 = pd.DataFrame(Table)
        DFL = DFL.append(DF1, sort=False)
    elif file.startswith('M'):
        Table2 = pd.read_csv(file, sep=',')
        DF2 = pd.DataFrame(Table2)
        DFM = DFM.append(DF2, sort=False)

Initializing Dialogue... 
Please select a file.


In [4]:
# Deklaruje listę kolumn którę będą mi potrzebne
DFL['NMIOT'] = 1
cols1 = ['LPROS1', 'LRASA1', 'LDUR', 'LDOP', 'LRASAOJ', 'LIL11', 'LIL21','LWAGA1','M_WAGUR','LDBRAK','NMIOT','LREJ1',
        'LWOJ1', 'LSEK1', 'LSTA1','NROJM1','LSUT1', 'NRMAM1', 'NOJMIO']
DFL = DFL[cols1]

In [5]:
# Zmieniam nazwy kolumn tak by odpowiadały kolumnon z DFL
DFM['LSUT1'] = np.nan  
DFM = DFM.rename(columns={'LDUR': 'LDUR1'})
DFM = DFM.rename(columns={'LDOP': 'LDUR'})
DFM = DFM.rename(columns={'LDUR1': 'LDOP'})
DFM = DFM.rename(columns={'RASAOJ': 'LRASAOJ'})
DFM = DFM.rename(columns={'NRO': 'NROJM1'})
DFM = DFM.rename(columns={'LREJ': 'LREJ1'})
DFM = DFM.rename(columns={'LWOJ': 'LWOJ1'})
# Deklaruje listę kolumn którę będą mi potrzebne
cols2 = ['LPROS1', 'LRASA1', 'LDUR', 'LDOP', 'LRASAOJ', 'LIL11', 'LIL21','LWAGA1','M_WAGUR','LDBRAK','NMIOT','LREJ1',
        'LWOJ1', 'LSEK1', 'LSTA1','LSUT1','NROJM1']
DFM = DFM[cols2]

In [6]:
#LLLLL
#NOJMIO - numer ojca miotu pierwszego
#NROJM1, MRMAM1 - rodzice LPROS1 
#LDUR - DURL - data urodzenia lochy
#LDOP - DURM - data urodzenia miotu
#MMMMM
#NRO - numer ojca miotu kolejnego

In [7]:
# Rozdzielam sutki prawe od lewych
DFL['PSUT'] = DFL['LSUT1'].astype(str).str[1]
DFL['LSUT1'] = DFL['LSUT1'].astype(str).str[0]

In [8]:
# Convertuje liczby sutków do inta
convert_dict = {'LSUT1': int,
                'PSUT': int
               }
  
DFL = DFL.astype(convert_dict)

In [9]:
# Liczę sumę sutków prawych i lewych
DFL['ISUT'] = DFL['PSUT'] + DFL['LSUT1']

In [10]:
# Wprowadzamy przedział czasowy z jakiego będą pobieranie dane
date1 = input('Date range from YYYY-MM-DD') 
date2 = input('Date range to YYYY-MM-DD')

Date range from YYYY-MM-DD 2019-12-12
Date range to YYYY-MM-DD 2020-12-11


In [11]:
# Funkcja zamienia format kolumn na format daty
class ToDate():

    def change_to_date(self, a, b):
        a[b] = pd.to_datetime(a[b])


dat = ToDate()
dat.change_to_date(DFL, 'LDOP')
dat.change_to_date(DFM, 'LDOP')
dat.change_to_date(DFL, 'LDUR')
dat.change_to_date(DFM, 'LDUR')

In [12]:
# W DF zostawaimy tylko dane z wcześniej zadeklarowanego przedziału czasowego
DFL = DFL[(DFL['LDOP'] >= date1) & (DFL['LDOP'] <= date2)]
DFM = DFM[(DFM['LDOP'] >= date1) & (DFM['LDOP'] <= date2)]

In [13]:
# Wyliczamy:
# Datę pierwszego oprosienia
DFL['POP'] = DFL['LDOP'] - DFL['LDUR']
# Okres międzymiotu
DFM['OMM'] = DFM['LDOP'] - DFM['LDUR']
# Zamieniam ww wartości na dni
DFL['POP'] = DFL['POP'] / np.timedelta64(1, 'D')
DFM['OMM'] = DFM['OMM'] / np.timedelta64(1, 'D')

In [14]:
# W zmiennej DF3 łączymy poprzednie DF
DF3 = DFL.append(DFM)

In [15]:
DF3 = DF3.rename(columns={'LDUR': 'DURL'})
DF3 = DF3.rename(columns={'LDOP': 'DURM'})

In [16]:
# Wyliczamy ile prosiąt nie dożyło 21 dnia życia
DF3['STRATY'] = DF3['LIL11'] - DF3['LIL21']

In [17]:
# Wyliczamy wartość % strat
DF3['ST21'] = (DF3['STRATY'] / DF3['LIL11']) * 100

In [19]:
# Funkcja tworzy DF z określonymi już kolumnami
def createDT(name):
    temp = pd.DataFrame(columns = ['Rasa','liczba loch','liczba ocenionych miotów ogółem', 
                                   'liczba ocenionych miotów pierw.','Liczba prosiąt urodzonych w miocie',
                                   'Liczba prosiąt w 21 dniu życia','Straty prosiąt do 21 dnia życia','Liczba sutków lochy',
                                   'Wiek w dniu pierwszego oprosienia(dni)','Okres miedzymiotu(dni)'],index =[1])
    temp['Rasa'] = name
    return temp

In [20]:
# Funkcja sortuje dane po odpowiedniej rasie
def sortedDT(DT, col, number):
    newDT = DT[DT[col] == number]
    return newDT

def sorted2DT(DT,number1,number2):
    newDT = DT[(DT.LRASA1 == number1) & (DT.LRASAOJ == number2)]
    return newDT

def months(DT, col, col2):
    newDT = DT[DT[col] != DT[col2]]
    return newDT


'''def birth(DFL, DFM):
    if len(DFL) > 0:
        pigs = DFL[['LIL11', 'LIL21']]
        pigs2 = DFM[['LIL11', 'LIL21']]
        pigs = pigs.append(pigs2)
        pigs['straty'] = pigs['LIL11'] - pigs['LIL21']
        pigs['ST21'] = (pigs['straty'] / pigs['LIL11']) * 100
        return pigs
    else:
        pigs = DFM[['LIL11', 'LIL21']]
        pigs['straty'] = pigs['LIL11'] - pigs['LIL21']
        pigs['ST21'] = (pigs['straty'] / pigs['LIL11']) * 100
        return pigs

    return s'''

"def birth(DFL, DFM):\n    if len(DFL) > 0:\n        pigs = DFL[['LIL11', 'LIL21']]\n        pigs2 = DFM[['LIL11', 'LIL21']]\n        pigs = pigs.append(pigs2)\n        pigs['straty'] = pigs['LIL11'] - pigs['LIL21']\n        pigs['ST21'] = (pigs['straty'] / pigs['LIL11']) * 100\n        return pigs\n    else:\n        pigs = DFM[['LIL11', 'LIL21']]\n        pigs['straty'] = pigs['LIL11'] - pigs['LIL21']\n        pigs['ST21'] = (pigs['straty'] / pigs['LIL11']) * 100\n        return pigs\n\n    return s"

In [21]:
'''hybryd1 = DF3['LRASA1']
hybryd2 = DF3['LRASAOJ']
nonHybryd = hybryd1 == hybryd2
hybryd = hybryd1!=hybryd2
DF4 = DF3[hybryd]
DF3 = DF3[nonHybryd]'''


"hybryd1 = DF3['LRASA1']\nhybryd2 = DF3['LRASAOJ']\nnonHybryd = hybryd1 == hybryd2\nhybryd = hybryd1!=hybryd2\nDF4 = DF3[hybryd]\nDF3 = DF3[nonHybryd]"

In [22]:
#'liczba ocenionych miotów ogółem' = len(DF3)
# 'liczba ocenionych miotów pierw.' = len (DF3)'NMIOT == 1 '
# liczba loch = LRPOS1.drop_duplicate()
# miedzymioty = nowa kolumna

In [23]:
# Funkcja wylicza wszystkie niezbędne wartości i przypisuje je do DF
def all_count(DF3, new):
    new['liczba loch'] = len(DF3.drop_duplicates(subset='LPROS1', keep="first").copy())
    new['liczba ocenionych miotów ogółem'] = len(DF3)
    new['liczba ocenionych miotów pierw.'] = len(DF3[DF3['NMIOT']==1])
    new['Liczba prosiąt urodzonych w miocie'] = np.around(DF3['LIL11'].mean(), decimals=2)
    new['Liczba prosiąt w 21 dniu życia'] = np.around(DF3['LIL21'].mean(), decimals=2)
    new['Straty prosiąt do 21 dnia życia'] = np.around(DF3['ST21'].mean(), decimals=2)
    new['Wiek w dniu pierwszego oprosienia(dni)'] = np.around(DF3['POP'].mean())
    new['Okres miedzymiotu(dni)'] = np.around(DF3['OMM'].mean())
    new['Liczba sutków lochy'] = np.around(DF3['ISUT'].mean(), decimals=2)

    return new

In [24]:

HAMP = createDT('Hampshire')
hamp = sortedDT(DF3,'LRASA1',60).copy()
all_count(hamp,HAMP)

,Rasa,liczba loch,liczba ocenionych miotów ogółem,liczba ocenionych miotów pierw.,Liczba prosiąt urodzonych w miocie,Liczba prosiąt w 21 dniu życia,Straty prosiąt do 21 dnia życia,Liczba sutków lochy,Wiek w dniu pierwszego oprosienia(dni),Okres miedzymiotu(dni)
1,Hampshire,13,26,5,12.42,11.04,11.15,14.2,445.0,182.0


In [55]:
WBP = createDT('Wbp')
wbp = sortedDT(DF3,'LRASA1',10).copy()
all_count(wbp,WBP)

PBZ = createDT('Pbz')
pbz = sortedDT(DF3,'LRASA1',20).copy()
all_count(pbz,PBZ)

HAMP = createDT('Hampshire')
hamp = sortedDT(DF3,'LRASA1',60).copy()
all_count(hamp,HAMP)

DUR = createDT('Duroc')
dur = sortedDT(DF3,'LRASA1',70).copy()
all_count(dur,DUR)

PIET = createDT('Pietrain')
piet = sortedDT(DF3,'LRASA1',80).copy()
all_count(piet,PIET)

L99 = createDT('Linia 990')
l99 = sortedDT(DF3,'LRASA1',90).copy()
all_count(l99,L99)

ZB = createDT('Złotnicka Biała')
zb = sortedDT(DF3,'LRASA1',30).copy()
all_count(zb,ZB)

ZPS = createDT('Złotnicka Pstra')
zps = sortedDT(DF3,'LRASA1',50).copy()
all_count(zps,ZPS)

PULA = createDT('Puławska')
pula = sortedDT(DF3,'LRASA1',40).copy()
all_count(pula,PULA)

,Rasa,liczba loch,liczba ocenionych miotów ogółem,liczba ocenionych miotów pierw.,Liczba prosiąt urodzonych w miocie,Liczba prosiąt w 21 dniu życia,Straty prosiąt do 21 dnia życia,Liczba sutków lochy,Wiek w dniu pierwszego oprosienia(dni),Okres miedzymiotu(dni)
1,Puławska,2110,3316,590,10.29,9.14,10.64,14.34,378.0,202.0


In [35]:
last = pd.concat([WBP,PBZ,HAMP,DUR,PIET,L99,PULA,ZB,ZPS])
last.set_index('Rasa',inplace=True)
last

,liczba loch,liczba ocenionych miotów ogółem,liczba ocenionych miotów pierw.,Liczba prosiąt urodzonych w miocie,Liczba prosiąt w 21 dniu życia,Straty prosiąt do 21 dnia życia,Liczba sutków lochy,Wiek w dniu pierwszego oprosienia(dni),Okres miedzymiotu(dni)
Rasa,,,,,,,,,
Wbp,1944,3315,736,12.48,11.65,6.26,14.99,365.0,167.0
Pbz,3104,5134,1287,12.65,11.72,6.82,15.29,359.0,173.0
Hampshire,13,26,5,12.42,11.04,11.15,14.20,445.0,182.0
Duroc,328,556,103,11.87,10.82,8.26,13.66,371.0,180.0
Pietrain,298,522,95,11.58,10.90,5.54,14.11,383.0,181.0
Linia 990,895,1580,335,9.96,9.48,4.78,14.19,374.0,155.0
Puławska,2110,3316,590,10.29,9.14,10.64,14.34,378.0,202.0
Złotnicka Biała,1134,1627,208,8.62,8.00,6.44,14.00,384.0,235.0
Złotnicka Pstra,998,1272,167,8.24,6.98,14.56,13.24,426.0,262.0


In [28]:
wbp_wwa = sortedDT(wbp, 'LWOJ1',2)
wbp_bia = sortedDT(wbp, 'LWOJ1',4)
wbp_byd = sortedDT(wbp, 'LWOJ1',6)
wbp_gda = sortedDT(wbp, 'LWOJ1',8)
wbp_kta = sortedDT(wbp, 'LWOJ1',10)
wbp_kie = sortedDT(wbp, 'LWOJ1',12)
wbp_kos = sortedDT(wbp, 'LWOJ1',14)
wbp_krk = sortedDT(wbp, 'LWOJ1',16)
wbp_lbl = sortedDT(wbp, 'LWOJ1',18)
wbp_ldz = sortedDT(wbp, 'LWOJ1',20)
wbp_ols = sortedDT(wbp, 'LWOJ1',22)
wbp_opo = sortedDT(wbp, 'LWOJ1',24)
wbp_poz = sortedDT(wbp, 'LWOJ1',26)
wbp_rze = sortedDT(wbp, 'LWOJ1',28)
wbp_wrc = sortedDT(wbp, 'LWOJ1',32)
wbp_zgo = sortedDT(wbp, 'LWOJ1',32)

In [46]:
wbp_wwa2 = create_cityDT('Warszawa','wbp')
wbp_bia2 = create_cityDT('Białystok','wbp')
wbp_byd2 = create_cityDT('Bydgoszcz','wbp')
wbp_gda2 = create_cityDT('Gdańsk','wbp')
wbp_kta2 = create_cityDT('Katowice','wbp')
wbp_kie2 = create_cityDT('Kielce','wbp')
wbp_kos2 = create_cityDT('Koszalin','wbp')
wbp_krk2 = create_cityDT('Kraków','wbp')
wbp_lbl2 = create_cityDT('Lublin','wbp')
wbp_ldz2 = create_cityDT('Łódź','wbp')
wbp_ols2 = create_cityDT('Olsztyn','wbp')
wbp_opo2 = create_cityDT('Opole','wbp')
wbp_poz2 = create_cityDT('Poznań','wbp')
wbp_rze2 = create_cityDT('Rzeszów','wbp')
wbp_wrc2 = create_cityDT('Wrocław','wbp')
wbp_zgo2 = create_cityDT('Zielona Góra','wbp')

In [157]:
def create_cityDT(city, race):
    temp = pd.DataFrame(columns = ['Rasa', 'Miasto','liczba loch','liczba ocenionych miotów ogółem', 
                                   'liczba ocenionych miotów pierw.','Liczba prosiąt urodzonych w miocie',
                                   'Liczba prosiąt w 21 dniu życia','Straty prosiąt do 21 dnia życia','Liczba sutków lochy',
                                   'Wiek w dniu pierwszego oprosienia(dni)','Okres miedzymiotu(dni)'],index =[1])
    temp['Rasa'] = race
    temp['Miasto'] = city
    return temp

cityDF = pd.DataFrame(columns = ['Rasa', 'Miasto','liczba loch','liczba ocenionych miotów ogółem', 
                                   'liczba ocenionych miotów pierw.','Liczba prosiąt urodzonych w miocie',
                                   'Liczba prosiąt w 21 dniu życia','Straty prosiąt do 21 dnia życia','Liczba sutków lochy',
                                   'Wiek w dniu pierwszego oprosienia(dni)','Okres miedzymiotu(dni)'],index =[1])

temp =  pd.DataFrame(columns = ['Rasa', 'Miasto','liczba loch','liczba ocenionych miotów ogółem', 
                                   'liczba ocenionych miotów pierw.','Liczba prosiąt urodzonych w miocie',
                                   'Liczba prosiąt w 21 dniu życia','Straty prosiąt do 21 dnia życia','Liczba sutków lochy',
                                   'Wiek w dniu pierwszego oprosienia(dni)','Okres miedzymiotu(dni)'],index =[1])

temp_dic = {}
z = 0
for i in race_list:
        
    for x in dic_miasta:
        tempx = sortedDT(i, 'LWOJ1', x)
        
        tempy = create_cityDT(dic_miasta[x], race_names[z])
        all_count(tempx, tempy)
        temp_dic[x]=tempy
        if z < 8:
            z+=1

In [ ]:
temp_dict = {}

for i in race_list:
    for x in dic_miasta:
        tempa = sortedDT

In [149]:
temp_dic = {}
for i in dic_miasta:
    for x in race_names:
        tempy = create_cityDT(dic_miasta[i], race_names[z])
    for z in race_list:
        tempx = sortedDT(z, 'LWOJ1', i)
        all_count(tempx, tempy)
        temp_dic[i] = tempy

IndexError: list index out of range

In [109]:
# Deklaruje słoknik miast
dic_miasta = {2:'Warszawa',
             4:'Białystok',
             6:'Bydgoszcz',
             8:'Gdańsk',
             10:'Katowice',
             12:'Kielce',
             14:'Koszalin',
             16:'Kraków',
             18:'Lublin',
             20:'Łódź',
             22:'Olsztyn',
             24:'Opole',
             26:'Poznań',
             28:'Rzeszów',
             32:'Wrocław',
             34:'Zielona Góra'}
# Deklaruje listę wydzielonych ras
race_list = [wbp,pbz,hamp,dur,piet,l99,zb,zps,pula]
race_names = ['WBP','PBZ','Hampshire','Duroc','Pietrain','Linia 990','PUL','ZLB','ZLP']

In [101]:
all_count(wbp_wwa,wbp_wwa2)
all_count(wbp_bia,wbp_bia2)
all_count(wbp_byd,wbp_byd2)
all_count(wbp_gda,wbp_gda2)
all_count(wbp_kta,wbp_kta2)
all_count(wbp_kie,wbp_kie2)
all_count(wbp_kos,wbp_kos2)
all_count(wbp_krk,wbp_krk2)
all_count(wbp_lbl,wbp_lbl2)
all_count(wbp_ldz,wbp_ldz2)
all_count(wbp_ols,wbp_ols2)
all_count(wbp_opo,wbp_opo2)
all_count(wbp_poz,wbp_poz2)
all_count(wbp_rze,wbp_rze2)
all_count(wbp_wrc,wbp_wrc2)
all_count(wbp_zgo,wbp_zgo2)
wbp_rze2

,Rasa,Miasto,liczba loch,liczba ocenionych miotów ogółem,liczba ocenionych miotów pierw.,Liczba prosiąt urodzonych w miocie,Liczba prosiąt w 21 dniu życia,Straty prosiąt do 21 dnia życia,Liczba sutków lochy,Wiek w dniu pierwszego oprosienia(dni),Okres miedzymiotu(dni)
1,wbp,Rzeszów,58,106,16,12.59,12.02,4.43,15.06,350.0,158.0


In [50]:
all_of_cr = pd.concat([wbp_wwa2, wbp_bia2, wbp_byd2, wbp_gda2, wbp_kta2, wbp_kie2, wbp_kos2, wbp_krk2, wbp_lbl2, wbp_ldz2, wbp_ols2, wbp_opo2, wbp_poz2, wbp_rze2, wbp_wrc2, wbp_zgo2])
all_of_cr.set_index(['Rasa','Miasto'])

liczba loch  liczba ocenionych miotów ogółem  \
Rasa Miasto                                                       
wbp  Warszawa              148                              245   
     Białystok             666                             1189   
     Bydgoszcz              79                              129   
     Gdańsk                  0                                0   
     Katowice               66                               97   
     Kielce                  0                                0   
     Koszalin               30                               46   
     Kraków                 53                               92   
     Lublin                 80                              145   
     Łódź                   53                              104   
     Olsztyn                79                              113   
     Opole                   0                                0   
     Poznań                  0                                0   
     Rzeszów                58                              106   
     Wrocław                 0                                0   
     Zielona Góra            0                                0   

                   liczba ocenionych miotów pierw.  \
Rasa Miasto                                          
wbp  Warszawa                                   68   
     Białystok                                 242   
     Bydgoszcz                                  36   
     Gdańsk                                      0   
     Katowice                                   38   
     Kielce                                      0   
     Koszalin                                   14   
     Kraków                                     15   
     Lublin                                     38   
     Łódź                                        1   
     Olsztyn                                    42   
     Opole                                       0   
     Poznań                                      0   
     Rzeszów                                    16   
     Wrocław                                     0   
     Zielona Góra                                0   

                   Liczba prosiąt urodzonych w miocie  \
Rasa Miasto                                             
wbp  Warszawa                                   13.78   
     Białystok                                  12.41   
     Bydgoszcz                                  12.74   
     Gdańsk                                       NaN   
     Katowice                                   12.55   
     Kielce                                       NaN   
     Koszalin                                   13.15   
     Kraków                                     14.07   
     Lublin                                     11.64   
     Łódź                                       14.01   
     Olsztyn                                    12.06   
     Opole                                        NaN   
     Poznań                                       NaN   
     Rzeszów                                    12.59   
     Wrocław                                      NaN   
     Zielona Góra                                 NaN   

                   Liczba prosiąt w 21 dniu życia  \
Rasa Miasto                                         
wbp  Warszawa                               13.02   
     Białystok                              11.72   
     Bydgoszcz                              11.09   
     Gdańsk                                   NaN   
     Katowice                               10.54   
     Kielce                                   NaN   
     Koszalin                               11.15   
     Kraków                                 13.40   
     Lublin                                 10.41   
     Łódź                                   12.57   
     Olsztyn                                10.90   
     Opole                                    NaN   
     Poznań                                   NaN   
     Rzeszów     

In [29]:
pbz_wwa = sortedDT(pbz, 'LWOJ1',2)
pbz_kta = sortedDT(pbz, 'LWOJ1',10)
pbz_kta2 = createDT('pbz')
all_count(pbz_kta,pbz_kta2)

,Rasa,liczba loch,liczba ocenionych miotów ogółem,liczba ocenionych miotów pierw.,Liczba prosiąt urodzonych w miocie,Liczba prosiąt w 21 dniu życia,Straty prosiąt do 21 dnia życia,Liczba sutków lochy,Wiek w dniu pierwszego oprosienia(dni),Okres miedzymiotu(dni)
1,pbz,194,303,94,12.89,11.12,12.51,15.22,355.0,169.0


In [30]:
pbz_wwa2 = createDT('pbz')
all_count(pbz_wwa, pbz_wwa2)

,Rasa,liczba loch,liczba ocenionych miotów ogółem,liczba ocenionych miotów pierw.,Liczba prosiąt urodzonych w miocie,Liczba prosiąt w 21 dniu życia,Straty prosiąt do 21 dnia życia,Liczba sutków lochy,Wiek w dniu pierwszego oprosienia(dni),Okres miedzymiotu(dni)
1,pbz,37,70,14,14.47,13.83,3.96,15.79,401.0,154.0


In [31]:
wbp2 = createDT('wbp')

In [32]:
all_count(wbp_wwa, wbp2)

,Rasa,liczba loch,liczba ocenionych miotów ogółem,liczba ocenionych miotów pierw.,Liczba prosiąt urodzonych w miocie,Liczba prosiąt w 21 dniu życia,Straty prosiąt do 21 dnia życia,Liczba sutków lochy,Wiek w dniu pierwszego oprosienia(dni),Okres miedzymiotu(dni)
1,wbp,148,245,68,13.78,13.02,5.1,15.51,384.0,160.0


In [33]:
wbp2

,Rasa,liczba loch,liczba ocenionych miotów ogółem,liczba ocenionych miotów pierw.,Liczba prosiąt urodzonych w miocie,Liczba prosiąt w 21 dniu życia,Straty prosiąt do 21 dnia życia,Liczba sutków lochy,Wiek w dniu pierwszego oprosienia(dni),Okres miedzymiotu(dni)
1,wbp,148,245,68,13.78,13.02,5.1,15.51,384.0,160.0


In [52]:
all_count(wbp_bia, city_wbp2)

,Rasa,liczba loch,liczba ocenionych miotów ogółem,liczba ocenionych miotów pierw.,Liczba prosiąt urodzonych w miocie,Liczba prosiąt w 21 dniu życia,Straty prosiąt do 21 dnia życia,Liczba sutków lochy,Wiek w dniu pierwszego oprosienia(dni),Okres miedzymiotu(dni)
1,wbp,666,1189,242,12.41,11.72,5.3,14.99,366.0,169.0


In [34]:
def city_count(DT,dt):
    for i, sub in dt.groupby('LWOJ1'):
        DT.loc[dic_miasta[i]]['Liczba loch'] = sub.drop_duplicates(subset='LPROS1', keep="first")(len)
        DT.loc[dic_miasta[i]]['liczba ocenionych miotów ogółem'] = len(sub)
        DT.loc[dic_miasta[i]]['liczba ocenionych miotów pierw.'] = len(sub[sub['NMIOT']==1])
        DT.loc[dic_miasta[i]]['Liczba prosiąt urodzonych w miocie'] = np.around(sub['LIL11'].mean(), decimals=2)
        DT.loc[dic_miasta[i]]['Liczba prosiąt w 21 dniu życia'] = np.around(sub['LIL21'].mean(), decimals=2)
        DT.loc[dic_miasta[i]]['Straty prosiąt do 21 dnia życia'] = np.around(sub['ST21'].mean(), decimals=2)
        DT.loc[dic_miasta[i]]['Wiek w dniu pierwszego oprosienia(dni)'] = np.around(sub['POP'].mean())
        DT.loc[dic_miasta[i]]['Okres miedzymiotu(dni)'] = np.around(sub['OMM'].mean())
        DT.loc[dic_miasta[i]]['Liczba sutków lochy'] = np.around(sub['ISUT'].mean(), decimals=2)

In [51]:
def createDT2():
    temp = pd.DataFrame(columns = ['Miasto','liczba loch','liczba ocenionych miotów ogółem', 
                                   'liczba ocenionych miotów pierw.','Liczba prosiąt urodzonych w miocie',
                                   'Liczba prosiąt w 21 dniu życia','Straty prosiąt do 21 dnia życia','Liczba sutków lochy',
                                   'Wiek w dniu pierwszego oprosienia(dni)','Okres miedzymiotu(dni)'])
    temp['Miasto'] = ['Warszawa','Białystok','Bydgoszcz','Gdańsk','Katowice','Kielce','Koszalin','Kraków','Lublin','Łódź'
                     'Olsztyn','Opole','Poznań','Rzeszów','Wrocław','Zielona Góra']
    temp = pd.DataFrame(temp).set_index('Miasto')
    return temp

In [52]:
city_wbp2 = createDT2()

In [53]:
city_wbp

NameError: name 'city_wbp' is not defined

In [40]:
wbp.to_csv('Results.csv', header=True, index=False, sep=';', mode='w')